# Installation of Libraries

In [85]:
!pip install pyyaml==5.4.1
!pip install gensim
!pip install sentence-transformers
!pip install tensorflow-hub
!pip install transformers
!pip install spacymoji
!pip install spacy==3.0.5
!pip install ekphrasis
!pip install preprocessor
!pip install ttp
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 23.1 MB/s 


In [86]:
import spacy
import spacymoji
import re
import torch
import pickle
import bz2
import nltk
import torch
import random
import gensim
import itertools
import math
import json
import csv
import urllib.request
import datetime
import emoji
import ekphrasis
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import gensim.downloader as api
import torch.nn.functional as F
import tensorflow_hub as hub
import preprocessor as tweet_preprocessor
import sklearn

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split, StratifiedKFold, RandomizedSearchCV, RepeatedKFold
from sklearn.pipeline import make_pipeline
from os.path import join as pjoin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from tqdm import tqdm
from matplotlib.pyplot import figure
from collections import Counter
from scipy import spatial
from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader
from sentence_transformers import SentenceTransformer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer
from dateutil.parser import parse
from ttp import ttp
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from scipy.stats import loguniform
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
base_dir = 'drive/MyDrive/CASCS505_FinalProj'

# Load datasets

In [ ]:
train = pd.read_json(pjoin(base_dir, 'train.zip'))
test = pd.read_json(pjoin(base_dir, 'test_questions.zip'))

In [ ]:
# full_data = pd.read_pickle('data.pkl')

In [ ]:
files.upload()

In [ ]:
train_vader = pd.read_csv('train_vader_sentiment.csv')
test_vader = pd.read_csv('test_vader_sentiment.csv')
train_glove = pd.read_csv('train_ids_glove.csv', header = None)
test_glove = pd.read_csv('test_ids_glove.csv', header = None)

In [ ]:
train_vader.columns = ['id', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']
test_vader.columns = ['id', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']

In [ ]:
train = train.set_index('id').join(train_vader.set_index('id'))
test = test.set_index('id').join(test_vader.set_index('id'))

In [ ]:
glove_train_val = train_glove.set_index(list(train_glove.columns[[0]])).values.tolist()
train_glove_lst = []
for i in glove_train_val:
  train_glove_lst.append(i)

glove_test_val = test_glove.set_index(list(test_glove.columns[[0]])).values.tolist()
test_glove_lst = []
for i in glove_test_val:
  test_glove_lst.append(i)

In [ ]:
train_glove = train_glove.set_index(list(train_glove.columns[[0]]))
test_glove = test_glove.set_index(list(test_glove.columns[[0]]))

In [ ]:
train_glove_modified = pd.DataFrame()
train_glove_modified['id'] = train_glove.index
train_glove_modified['glove'] = train_glove_lst

In [ ]:
test_glove_modified = pd.DataFrame()
test_glove_modified['id'] = test_glove.index
test_glove_modified['glove'] = test_glove_lst

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
train = train.set_index('id').join(train_glove_modified.set_index('id'))
test = test.set_index('id').join(test_glove_modified.set_index('id'))

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

# Pre Processing - Kavina

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

def preprocess_tweets(dataframe, text_processor):
  processed_lst = dataframe['text'].str.lower()
  temp = []
  for s in processed_lst:
    s = (" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^\w\d\s\<\>]+", '', s)
    temp.append(s)
  return temp

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
train['processed'] = preprocess_tweets(train, text_processor = text_processor)
test['processed'] = preprocess_tweets(test, text_processor = text_processor)

In [ ]:
stop_words = stopwords.words('english')
train['processed'] = train['processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
train['processed'] = train['processed'].replace('_', '', regex=True)
test['processed'] = test['processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
test['processed'] = test['processed'].replace('_', '', regex=True)

In [ ]:
tt = TweetTokenizer()
train['tokenized'] = train['processed'].apply(tt.tokenize)
test['tokenized'] = test['processed'].apply(tt.tokenize)

In [ ]:
train['type'] = 'train'
test['type'] = 'test'

In [ ]:
train = train[train['retweet_count'].notnull()]
full_data = pd.concat([train, test], ignore_index=True, sort=False)

# Sentence Embeddings

## Glove - Pujan

## TFIDF - Joel

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features = 100)
tfidf_vector = tfidf_vectorizer.fit_transform(full_data.processed)

In [ ]:
tfidf_lst = []
train_lst_tfidf = tfidf_vector[:len(train)] 
for idx, i in enumerate(train_lst_tfidf):
  tfidf_lst.append(train_lst_tfidf[idx])

train['tfidf'] = tfidf_lst

In [ ]:
tfidf_lst = []
test_lst_tfidf = tfidf_vector[len(train):] 
for idx, i in enumerate(test_lst_tfidf):
  tfidf_lst.append(test_lst_tfidf[idx])

In [ ]:
tfidf_lst = []
test_lst_tfidf = tfidf_vector[len(train):] 
for idx, i in enumerate(test_lst_tfidf):
  tfidf_lst.append(test_lst_tfidf[idx])

test['tfidf'] = tfidf_lst

## Count Vectorizer - Joel

In [ ]:
count_vectorizer = CountVectorizer(max_features = 100)
cv_vector = count_vectorizer.fit_transform(full_data.processed)

In [ ]:
cv_lst = []
train_lst_cv = cv_vector[:len(train)] 
for idx, i in enumerate(train_lst_cv):
  cv_lst.append(train_lst_cv[idx])

train['cv'] = cv_lst

In [ ]:
cv_lst = []
test_lst_cv = cv_vector[len(train):] 
for idx, i in enumerate(test_lst_cv):
  cv_lst.append(test_lst_cv[idx])

test['cv'] = cv_lst

## Bert - Joel

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
sbert_sentence_embeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  sbert_sentence_embeddings.append(sbert_model.encode(sentence))

train['bert'] = sbert_sentence_embeddings

100%|██████████| 96562/96562 [3:28:46<00:00,  7.71it/s]


In [ ]:
sbert_sentence_embeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  sbert_sentence_embeddings.append(sbert_model.encode(sentence))

test['bert'] = sbert_sentence_embeddings

100%|██████████| 32210/32210 [1:06:48<00:00,  8.04it/s]


## Doc2Vec - Joel

In [ ]:
tokenized_sent = []
for s in tqdm(full_data['processed'].tolist(), total = len(full_data)):
  tokenized_sent.append(word_tokenize(s.lower()))

100%|██████████| 128772/128772 [00:18<00:00, 6845.36it/s]


In [ ]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]

In [ ]:
d2v_model = Doc2Vec(tagged_data, vector_size = 100, window = 2, min_count = 1, epochs = 100)

In [ ]:
d2v_sentence_emebeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  doc = word_tokenize(sentence.lower())
  d2v_sentence_emebeddings.append(d2v_model.infer_vector(doc))

train['d2v'] = d2v_sentence_emebeddings

100%|██████████| 96562/96562 [08:15<00:00, 194.96it/s]


In [ ]:
d2v_sentence_emebeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  doc = word_tokenize(sentence.lower())
  d2v_sentence_emebeddings.append(d2v_model.infer_vector(doc))

test['d2v'] = d2v_sentence_emebeddings

100%|██████████| 32210/32210 [02:46<00:00, 193.77it/s]


## Universal Sentence Encoder - Joel

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
use_sentence_embeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  use_sentence_embeddings.append(use_model([sentence]))

train['use'] = use_sentence_embeddings

100%|██████████| 96562/96562 [05:44<00:00, 280.18it/s]


In [ ]:
use_sentence_embeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  use_sentence_embeddings.append(use_model([sentence]))

test['use'] = use_sentence_embeddings

100%|██████████| 32210/32210 [01:54<00:00, 280.71it/s]


# Sentiment

## Roberta - Joel

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
labels=[]

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
  html = f.read().decode('utf-8').split("\n")
  csvreader = csv.reader(html, delimiter='\t')

labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
auto_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
auto_model.save_pretrained(MODEL)

In [ ]:
auto_scores_lst = []
#tf_scores_lst = []
for sentence in tqdm(full_data['processed'].tolist(), total = len(full_data)):
  auto_encoded_input = tokenizer(sentence, return_tensors='pt')
  auto_output = auto_model(**auto_encoded_input)
  auto_scores = auto_output[0][0].detach().numpy()
  auto_scores = softmax(auto_scores)

  auto_scores_lst.append(auto_scores)

In [ ]:
full_data['auto_sentiment'] = auto_scores_lst

## Vader - Pujan

In [ ]:
def return_sentiments(idx,text):
    try:
        sid_obj = SentimentIntensityAnalyzer()
        # polarity_scores method of SentimentIntensityAnalyzer
        # object gives a sentiment dictionary.
        # which contains pos, neg, neu, and compound scores.
        sentiment_dict = sid_obj.polarity_scores(text)
        return (idx,sentiment_dict['neg'],sentiment_dict['neu'],sentiment_dict['pos'],sentiment_dict['compound'])
    except Exception as e:
        return(idx,-10,-10,-10,-10,-10)

# Feature Extraction - Akira

## Train

In [ ]:
custom_columns=["tweet_id", "timestamp", "year", "month", "week_day", "hour", "is_reply_to_tweet", "is_reply_to_user", "user_id", "@_count", "#_count", "url_count", "tweet_len", "rewteet_count", "favorite_count"]
features = []
for idx, twt in train.iterrows():
  row = [twt["id_str"]]
  time = twt['created_at']
  row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
  if twt["in_reply_to_status_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  if twt["in_reply_to_user_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  row.append(twt["user"]["id_str"])
  row.append(len(twt["entities"]["user_mentions"]))
  row.append(len(twt["entities"]["hashtags"]))
  row.append(len(twt["entities"]["urls"]))
  row.append(len(twt["text"]))
  row.append(twt["retweet_count"])
  row.append(twt["favorite_count"])

  features.append(row)

In [ ]:
feature_df = pd.DataFrame(data = features, columns = custom_columns)

In [ ]:
aggregate_columns=["tweet_id", "user_total_retweeted_so_far", "user_total_favorited_so_far", "user_total_retweeted", "user_total_favorited"]
aggregate_features=[]
for user_id, group in feature_df.groupby('user_id'):
  group_row = []
  total_retweet = 0
  total_favorite = 0
  for twt in group.iterrows():
    row = [twt[1]["tweet_id"]]
    total_retweet += twt[1]["rewteet_count"]
    total_favorite += twt[1]["favorite_count"]
    row.extend([total_retweet, total_favorite])
    group_row.append(row)
  for i in range(len(group_row)):
    group_row[i].extend([total_retweet, total_favorite])
  aggregate_features.extend(group_row)

In [ ]:
agg_df = pd.DataFrame(data = aggregate_features, columns = aggregate_columns)

In [ ]:
custom_features_df = pd.merge(feature_df, agg_df, on = "tweet_id")

In [ ]:
custom_features_df.reset_index(inplace=True, drop=True)
full_data.reset_index(inplace=True, drop=True)

In [ ]:
train = pd.concat([train, custom_features_df], axis = 1)

## Test

In [ ]:
custom_columns=["tweet_id", "timestamp", "year", "month", "week_day", "hour", "is_reply_to_tweet", "is_reply_to_user", "user_id", "@_count", "#_count", "url_count", "tweet_len"]
features = []
for idx, twt in test.iterrows():
  row = [twt["id_str"]]
  time = twt['created_at']
  row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
  if twt["in_reply_to_status_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  if twt["in_reply_to_user_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  row.append(twt["user"]["id_str"])
  row.append(len(twt["entities"]["user_mentions"]))
  row.append(len(twt["entities"]["hashtags"]))
  row.append(len(twt["entities"]["urls"]))
  row.append(len(twt["text"]))

  features.append(row)

In [ ]:
feature_df = pd.DataFrame(data = features, columns = custom_columns)

In [ ]:
feature_df.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [ ]:
test = pd.concat([test, feature_df], axis = 1)

# Save Data as pickle

In [ ]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

# Load Train and Test pkl

In [5]:
# train = pd.read_pickle(pjoin(base_dir, 'train.pkl'))
# test = pd.read_pickle(pjoin(base_dir, 'test.pkl'))

# Preprocessing - Joel

In [6]:
train = train[train['retweet_count'].notnull()]

In [7]:
train["is_reply_to_status_id"] = train["in_reply_to_status_id"].apply(lambda x: 1  if  np.isnan(x)  else  0 ) 
test["is_reply_to_status_id"] = test["in_reply_to_status_id"].apply(lambda x: 1  if  np.isnan(x)  else  0 ) 
train["is_reply_to_user_id"] = train["in_reply_to_user_id"].apply(lambda x: 1 if np.isnan(x)  else  0 ) 
test["is_reply_to_user_id"] = test["in_reply_to_user_id"].apply(lambda x: 1 if np.isnan(x)  else  0 ) 
train["reply_to_user_id_count"] = train.groupby("in_reply_to_user_id").in_reply_to_user_id.transform("count").fillna(0)
test["reply_to_user_id_count"] = test.groupby("in_reply_to_user_id").in_reply_to_user_id.transform("count").fillna(0)
train["reply_to_status_id_count"] = train.groupby("in_reply_to_status_id").in_reply_to_user_id.transform("count").fillna(0)
test["reply_to_status_id_count"] = test.groupby("in_reply_to_status_id").in_reply_to_user_id.transform("count").fillna(0)

In [8]:
col_to_remove=["contributors", "retweeted", "coordinates", "extended_entities", "geo", "place", "quoted_status", "retweeted_status"]
train.drop(col_to_remove, axis = 1,inplace = True)
test.drop(col_to_remove, axis = 1, inplace = True)
    
def parse_json(x):
  for col in ['entities', 'user'] :
    print(col)
    for key in x[col].iloc[0].keys():
      x[col+"_"+key] = x[col].apply(lambda x: x[key])
  return x 

train = parse_json(train)
test = parse_json(test)
json_columns = []
for col in train.columns : 
  try :
    a = train[col].unique()
  except : 
    json_columns.append(col)

entities
user
entities
user


In [9]:
cols = ["entities_symbols","entities_hashtags","entities_urls"]
for col in cols  : 
  train[col] = train[col].apply(lambda x : x if len(x)>0 else np.nan)
  test[col] = test[col].apply(lambda x : x if len(x)>0 else np.nan)  

In [10]:
col_to_remove = ["entities","id_str","in_reply_to_screen_name","in_reply_to_status_id_str","in_reply_to_user_id_str",
              "quoted_status_id_str","text","user","entities_symbols","user_time_zone","user_time_zone",
              "user_profile_sidebar_fill_color","user_profile_banner_url","user_is_translator",
              "user_has_extended_profile","user_translator_type","user_profile_link_color","user_screen_name",                                                        
               "user_profile_link_color","user_profile_background_image_url",
               "user_utc_offset","user_protected","user_profile_background_color",
              "user_geo_enabled","user_profile_image_url","user_profile_use_background_image","user_description",
               "user_profile_image_url_https","user_profile_background_tile","user_following","user_contributors_enabled",
               "user_id_str","user_name","user_profile_background_image_url_https","user_profile_sidebar_border_color",
               "user_default_profile_image","user_url","user_notifications","user_profile_text_color" ,"user_lang",
              "quoted_status_id","user_follow_request_sent","favorited"]

train["len_text"] = train.text.apply(lambda x :len(x.split(" ")))
test["len_text"] = train.text.apply(lambda x :len(x.split(" ")))

train.drop(col_to_remove, axis = 1,inplace = True)
test.drop(col_to_remove, axis = 1,inplace = True)


In [11]:
def map_categorical_feautres(columns=None,Data=[]):
  mapping_dict={}
  assert(isinstance(Data,list)) , "data must be  List"
  for col in columns : 
    print(col)
    unique = []
    for data in Data: 
        unique.extend(data[col].unique().tolist()) 
    unique = list(set(unique))
    mapp_dict = {}
    for i in range(len(unique)):
      mapp_dict[unique[i]] = i
    for data in Data:  
      data[col] = data[col].map(mapp_dict)
    mapping_dict[col] = mapp_dict

In [12]:
col_to_factorize=["user_location","user_id","user_default_profile","user_verified", "truncated", "source" , "lang", "is_quote_status","user_is_translation_enabled", 'year', 'month', 'week_day', 'hour']
map_categorical_feautres(Data = [train,test], columns = col_to_factorize)

user_location
user_id
user_default_profile
user_verified
truncated
source
lang
is_quote_status
user_is_translation_enabled
year
month
week_day
hour


In [13]:
train["count_entities_user_mentions"] = train["entities_user_mentions"].apply(len)
test["count_entities_user_mentions"] = test["entities_user_mentions"].apply(len)
train["count_entities_hashtags"] = train["entities_hashtags"].fillna("").apply(len)
test["count_entities_hashtags"] = test["entities_hashtags"].fillna("").apply(len)
train["count_entities_urls"] = train["entities_urls"].fillna("").apply(len)
test["count_entities_urls"] = test["entities_urls"].fillna("").apply(len)

In [14]:
train.possibly_sensitive = train.possibly_sensitive.fillna(0)

In [15]:
test.possibly_sensitive = test.possibly_sensitive.fillna(0)

In [16]:
train['retweet_count'] = np.log1p(train['retweet_count'])

In [ ]:
train_label = train['retweet_count']
not_used_cols = ['rewteet_count', 'user_total_retweeted_so_far', 'user_total_favorited_so_far', 'user_total_retweeted', 'user_total_favorited', 'glove', 'tfidf', 'cv', 'bert', 'd2v', 'use', 'processed', 'tokenized', 'type', 'entities_urls', 'entities_hashtags', 'entities_user_mentions', 'user_entities', 'favorite_count', 'user_created_at', 'retweet_count', "in_reply_to_user_id", "train", "in_reply_to_status_id", "id", "favorite_count", "created_at", "retweet_countpred", "retweet_count_without_zeros", "entities_hashtags_text"]
features_name = [f for f in train.columns if f not in not_used_cols]

In [17]:
#train.sort_values(by=['created_at'], inplace=True)
#train.reset_index(inplace=True)

In [18]:
#train = train.iloc[0:round(0.8*len(train))]
#validation = train.iloc[round(0.8*len(train)):]

# Linear Regression Model - Joel

## Glove

In [ ]:
glove_df = train.glove.apply(pd.Series)

In [ ]:
modified_train = train[features_name]

In [ ]:
train_glove = pd.concat([modified_train, glove_df], axis=1).iloc[: , :-1]

In [25]:
train_glove.set_index('tweet_id', inplace = True)

In [27]:
model = Ridge()
cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state = 1, verbose = 3)

result = search.fit(train_glove, train_label)

Fitting 9 folds for each of 8 candidates, totalling 72 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [29]:
glove_model = Ridge(**result.best_params_)
glove_model.fit(train_glove, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Ridge(fit_intercept=False, solver='svd')

In [30]:
test_glove_df = test.glove.apply(pd.Series)
test_modified = test[features_name]
test_glove = pd.concat([test_modified, test_glove_df], axis=1).iloc[: , :-1]
test_glove.set_index('tweet_id', inplace = True)
test_pred = glove_model.predict(test_glove)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [31]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_glove.csv', index = False)

In [32]:
# validation_label = validation['retweet_count']
# validation_glove_df = validation.glove.apply(pd.Series)
# validation = validation[features_name]
# validation_glove = pd.concat([validation, validation_glove_df], axis=1).iloc[: , :-1]
# validation_glove.set_index('tweet_id', inplace = True)
# validation_pred = glove_model.predict(validation_glove)

# import sklearn.metrics as metrics

# mae = metrics.mean_absolute_error(validation_label, validation_pred)
# mse = metrics.mean_squared_error(validation_label, validation_pred)
# rmse = np.sqrt(mse) 
# r2 = metrics.r2_score(validation_label, validation_pred)

# print("Results of sklearn.metrics:")
# print("MAE:",mae)
# print("MSE:", mse)
# print("RMSE:", rmse)
# print("R-Squared:", r2)

NameError: ignored

## Count Vectorizer

In [48]:
lst = []
for i in tqdm(train.cv, total = len(train)):
  lst.append(i.toarray()[0].tolist())

100%|██████████| 96562/96562 [00:00<00:00, 108290.27it/s]


In [49]:
cv_df = pd.DataFrame(lst)

In [50]:
modified_train = train[features_name]

In [51]:
train_cv = pd.concat([modified_train, cv_df], axis=1).iloc[: , :-1]

In [52]:
train_cv.set_index('tweet_id', inplace = True)

In [54]:
model = Ridge()
cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state = 1)

result = search.fit(train_cv, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.85517e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [55]:
cv_model = Ridge(**result.best_params_)
cv_model.fit(train_cv, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.85517e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


Ridge(solver='cholesky')

In [56]:
lst = []
for i in tqdm(test.cv, total = len(test)):
  lst.append(i.toarray()[0].tolist())

test_cv_df = pd.DataFrame(lst)
test_modified = test[features_name]
test_cv = pd.concat([test_modified, test_cv_df], axis=1).iloc[: , :-1]
test_cv.set_index('tweet_id', inplace = True)
test_pred = cv_model.predict(test_cv)

100%|██████████| 32210/32210 [00:00<00:00, 68789.40it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [57]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_cv.csv', index = False)

In [58]:
# validation_label = validation['retweet_count']
# lst = []
# for i in tqdm(validation.cv, total = len(validation)):
#   lst.append(i.toarray()[0].tolist())
# validation_cv_df = pd.DataFrame(lst)
# validation = validation[features_name]
# validation.reset_index(inplace = True)
# validation_cv = pd.concat([validation, validation_cv_df], axis=1).iloc[: , :-1]
# validation_cv.set_index('tweet_id', inplace = True)
# validation_cv.drop(columns = ['level_0'], inplace = True)
# validation_pred = cv_model.predict(validation_cv)

# import sklearn.metrics as metrics

# mae = metrics.mean_absolute_error(validation_label, validation_pred)
# mse = metrics.mean_squared_error(validation_label, validation_pred)
# rmse = np.sqrt(mse) 
# r2 = metrics.r2_score(validation_label, validation_pred)

# print("Results of sklearn.metrics:")
# print("MAE:",mae)
# print("MSE:", mse)
# print("RMSE:", rmse)
# print("R-Squared:", r2)

NameError: ignored

## TFIDF

In [96]:
lst = []
for i in tqdm(train.tfidf, total = len(train)):
  lst.append(i.toarray()[0].tolist())

100%|██████████| 96562/96562 [00:02<00:00, 40810.86it/s]


In [97]:
tfidf_df = pd.DataFrame(lst)

In [98]:
train_modified = train[features_name]

In [99]:
train_tfidf = pd.concat([train_modified, tfidf_df], axis=1).iloc[: , :-1]

In [100]:
train_tfidf.set_index('tweet_id', inplace = True)

In [101]:
model = Ridge()

cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state= 1 )

result = search.fit(train_tfidf, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.85034e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [102]:
tfidf_model = Ridge(**result.best_params_)
tfidf_model.fit(train_tfidf, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.85034e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


Ridge(solver='cholesky')

In [103]:
lst = []
for i in tqdm(test.tfidf, total = len(test)):
  lst.append(i.toarray()[0].tolist())

test_tfidf_df = pd.DataFrame(lst)
test_modified = test[features_name]
test_tfidf = pd.concat([test_modified, test_tfidf_df], axis=1).iloc[: , :-1]
test_tfidf.set_index('tweet_id', inplace = True)
test_pred = tfidf_model.predict(test_tfidf)

100%|██████████| 32210/32210 [00:01<00:00, 30374.35it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [104]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_tfidf.csv', index = False)

## Bert

In [22]:
bert_df = train.bert.apply(pd.Series)

In [23]:
modified_train = train[features_name]

In [24]:
train_bert = pd.concat([modified_train, bert_df], axis=1).iloc[: , :-1]

In [25]:
train_bert.set_index('tweet_id', inplace = True)

In [27]:
model = Ridge()

cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state = 1)

result = search.fit(train_bert, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.84568e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [28]:
bert_model = Ridge(**result.best_params_)
bert_model.fit(train_bert, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Ridge(fit_intercept=False, solver='sag')

In [29]:
test_bert_df = test.bert.apply(pd.Series)
test_modified = test[features_name]
test_bert = pd.concat([test_modified, test_bert_df], axis=1).iloc[: , :-1]
test_bert.set_index('tweet_id', inplace = True)
test_pred = bert_model.predict(test_bert)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [30]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_bert.csv', index = False)

## Doc2Vec

In [63]:
d2v_df = train.d2v.apply(pd.Series)

In [64]:
modified_train = train[features_name]

In [65]:
train_d2v = pd.concat([modified_train, d2v_df], axis=1).iloc[: , :-1]

In [66]:
train_d2v.set_index('tweet_id', inplace = True)

In [68]:
model = Ridge()

cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state = 1)

result = search.fit(train_d2v, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [69]:
d2v_model = Ridge(**result.best_params_)
d2v_model.fit(train_d2v, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Ridge(fit_intercept=False, solver='svd')

In [70]:
test_d2v_df = test.d2v.apply(pd.Series)
test_modified = test[features_name]
test_d2v = pd.concat([test_modified, test_d2v_df], axis=1).iloc[: , :-1]
test_d2v.set_index('tweet_id', inplace = True)
test_pred = d2v_model.predict(test_d2v)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [71]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_d2v.csv', index = False)

## USE

In [38]:
lst = []
for i in tqdm(train.use, total = len(train)):
  lst.append(i[0][0].numpy().tolist())

100%|██████████| 96562/96562 [00:47<00:00, 2045.59it/s]


In [39]:
use_df = pd.DataFrame(lst)

In [40]:
modified_train = train[features_name]

In [41]:
train_use = pd.concat([modified_train, use_df], axis=1).iloc[: , :-1]

In [42]:
train_use.set_index('tweet_id', inplace = True)

In [44]:
model = Ridge()

cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['fit_intercept'] = [True, False]
search = RandomizedSearchCV(model, space, n_iter = 500, scoring = 'neg_mean_absolute_error', n_jobs = -1, cv = cv, random_state = 1)

result = search.fit(train_use, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.99662e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [45]:
use_model = Ridge(**result.best_params_)
use_model.fit(train_use, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.99662e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


Ridge(fit_intercept=False, solver='cholesky')

In [46]:
lst = []
for i in tqdm(test.use, total = len(test)):
  lst.append(i[0][0].numpy().tolist())

test_use_df = pd.DataFrame(lst)
test_modified = test[features_name]
test_use = pd.concat([test_modified, test_use_df], axis=1).iloc[: , :-1]
test_use.set_index('tweet_id', inplace = True)
test_pred = use_model.predict(test_use)

100%|██████████| 32210/32210 [00:13<00:00, 2421.77it/s]


In [47]:
output_df = pd.DataFrame(test.id)
output_df['retweet_count'] = test_pred
output_df.columns = ['Id', 'retweet_count']
output_df['retweet_count']=np.expm1(output_df["retweet_count"])
output_df[output_df < 0] = 0
output_df.to_csv('retweet_prediction_d2v.csv', index = False)